Try to add a relationship to temperature extremes

In [1]:
import utils
import json
import plotly.express as px


with open('sub_codelists.json', 'r') as f:
    sub_codelists = json.load(f)


In [2]:
average_household_constraints = {
    'freq': 'A',
    'unit': 'AVG',
    'geo': sub_codelists['geo_eu_countries']
}

nrg_bal_constraints = {
    'freq': 'A',
    'nrg_bal': 'FC_OTH_HH_E',
    'siec': 'TOTAL',
    'unit': 'GWH',
    'geo': sub_codelists['geo_eu_countries'],
}

demo_pjan_constraints = {
    'freq': 'A',
    'unit': 'NR',
    'age': 'TOTAL',
    'sex': 'T',
    'geo': sub_codelists['geo_eu_countries'],
    }

In [3]:
average_household = utils.get_dataset_with_selection('ilc_lvph01', average_household_constraints)

demo_pjan = utils.get_dataset_with_selection('demo_pjan', demo_pjan_constraints)
demo_pjan.data.TIME_PERIOD  = demo_pjan.data.TIME_PERIOD  - 1

nrg_bal = utils.get_dataset_with_selection('nrg_bal_s', nrg_bal_constraints)


data_url: https://ec.europa.eu/eurostat/api/dissemination/sdmx/2.1/data/ilc_lvph01/A.AVG.BE+BG+CZ+DK+DE+EE+IE+EL+ES+FR+HR+IT+CY+LV+LT+LU+HU+MT+NL+AT+PL+PT+RO+SI+SK+FI+SE?format=SDMX-CSV&startPeriod=2000
metadata_url: https://ec.europa.eu/eurostat/api/dissemination/sdmx/2.1/dataflow/all/ilc_lvph01/latest?detail=full&references=descendants
data_url: https://ec.europa.eu/eurostat/api/dissemination/sdmx/2.1/data/demo_pjan/A.NR.TOTAL.T.BE+BG+CZ+DK+DE+EE+IE+EL+ES+FR+HR+IT+CY+LV+LT+LU+HU+MT+NL+AT+PL+PT+RO+SI+SK+FI+SE?format=SDMX-CSV&startPeriod=2000
metadata_url: https://ec.europa.eu/eurostat/api/dissemination/sdmx/2.1/dataflow/all/demo_pjan/latest?detail=full&references=descendants
data_url: https://ec.europa.eu/eurostat/api/dissemination/sdmx/2.1/data/nrg_bal_s/A.FC_OTH_HH_E.TOTAL.GWH.BE+BG+CZ+DK+DE+EE+IE+EL+ES+FR+HR+IT+CY+LV+LT+LU+HU+MT+NL+AT+PL+PT+RO+SI+SK+FI+SE?format=SDMX-CSV&startPeriod=2000
metadata_url: https://ec.europa.eu/eurostat/api/dissemination/sdmx/2.1/dataflow/all/nrg_bal_s/l

In [4]:
demo_pjan_df = demo_pjan.data.drop(columns=['OBS_FLAG'])
demo_pjan_df.set_index(['geo', 'TIME_PERIOD'], inplace=True)

average_household_df = average_household.data.drop(columns=['OBS_FLAG'])
average_household_df.set_index(['geo', 'TIME_PERIOD'], inplace=True)

number_of_households_df = demo_pjan_df.div(average_household_df)
display(number_of_households_df)


OBS_VALUE
geo TIME_PERIOD              
AT  1999                  NaN
    2000                  NaN
    2001                  NaN
    2002                  NaN
    2003         3.392739e+06
...                       ...
SK  2019         1.882025e+06
    2020         1.882683e+06
    2021         1.874039e+06
    2022         1.751223e+06
    2023         1.749899e+06

[682 rows x 1 columns]

In [5]:
nrg_bal_df = nrg_bal.data.set_index(['geo', 'TIME_PERIOD'])

display(nrg_bal.data)

,geo,TIME_PERIOD,OBS_VALUE
0,AT,2000,73830.480
1,AT,2001,77829.265
2,AT,2002,74848.515
3,AT,2003,76322.168
4,AT,2004,74796.391
...,...,...,...
643,SK,2019,30676.211
644,SK,2020,31919.356
645,SK,2021,34498.237
646,SK,2022,31244.142


In [6]:
consumption_per_household = nrg_bal_df.div(number_of_households_df)

In [7]:
consumption_per_household.reset_index(inplace=True)
consumption_per_household['OBS_VALUE'] = consumption_per_household['OBS_VALUE'] * 1000
consumption_per_household.rename(columns={'OBS_VALUE': 'consumption_per_household_MWh'}, inplace=True)

fig = px.line(consumption_per_household, x='TIME_PERIOD', y='consumption_per_household_MWh', color='geo')
fig.show()
